In [ ]:
import tensorflow as tf
import time

a = tf.random.normal((3000, 3000))
b = tf.random.normal((3000, 3000))

start = time.time()
c = tf.matmul(a, b)
tf.print("Time:", time.time() - start)


In [ ]:
# ============================================================
#   SINGLE-CELL: High Accuracy + Fast Training + Local ResNet50
# ============================================================

# ---------------- USER SETTINGS ----------------
ENCODER = "resnet50"        # Best accuracy
USE_LOCAL_RESNET = True
LOCAL_RESNET_DIR = "/kaggle/input/resnet50-pretrained-weights"
LOCAL_RESNET_WEIGHTS = LOCAL_RESNET_DIR + "/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"

COCO_DIR = "/kaggle/input/coco-2017-dataset"
MAX_IMAGES = 40000
IMG_SIZE = (256, 256)
BATCH = 8
EPOCHS = 8
SEED = 42

AUTO_WRITE_TFRECORD = True
TRAIN_NOW = True
# ------------------------------------------------

# ================= ENVIRONMENT SAFE =================
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_XLA_FLAGS"] = "--tf_xla_enable_xla_devices=false"

# ================= IMPORTS =================
import gc, math, time
from pathlib import Path
import numpy as np
import cv2
from tqdm import tqdm

import tensorflow as tf
print("TensorFlow:", tf.__version__)

gpus = tf.config.list_physical_devices("GPU")
print("GPUs detected:", gpus)
# ❌ DO NOT set memory growth — Kaggle initializes TF early

from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")
print("Mixed precision ON")

# ================= PATHS =================
BASE_INPUT = Path(COCO_DIR) / "coco2017"
IMG_DIR = BASE_INPUT / "train2017"
ANN_PATH = BASE_INPUT / "annotations" / "instances_train2017.json"

OUT = Path("/kaggle/working/coco_processed")
RGB_DIR = OUT / "rgb_256"
GRAY_DIR = OUT / "gray_256"
MASK_DIR = OUT / "masks_256"
TF_DIR = Path("/kaggle/working/tfrecords")

TF_DIR.mkdir(parents=True, exist_ok=True)
for d in [OUT, RGB_DIR, GRAY_DIR, MASK_DIR]:
    d.mkdir(parents=True, exist_ok=True)

TRAIN_TF = TF_DIR / "train.tfrecord"
VAL_TF   = TF_DIR / "val.tfrecord"

print("COCO found:", IMG_DIR.exists(), ANN_PATH.exists())
print("Local ResNet weights found:", Path(LOCAL_RESNET_WEIGHTS).exists())

# ================= LOAD COCO =================
from pycocotools.coco import COCO
coco = COCO(str(ANN_PATH))
all_ids = coco.getImgIds()
image_ids = all_ids[:MAX_IMAGES]
print("Images used:", len(image_ids))

IMG_H, IMG_W = IMG_SIZE

# ================= STEP 1: MASKED RGB =================
if len(list(RGB_DIR.glob("*.jpg"))) < len(image_ids):
    print("Creating masked RGB images...")
    for iid in tqdm(image_ids):
        out = RGB_DIR / f"{iid}.jpg"
        if out.exists():
            continue
        info = coco.loadImgs([iid])[0]
        img = cv2.imread(str(IMG_DIR / info["file_name"]))
        if img is None: 
            continue

        h, w = img.shape[:2]
        ann_ids = coco.getAnnIds(imgIds=iid)
        mask = np.zeros((h, w), dtype=np.uint8)
        for ann in coco.loadAnns(ann_ids):
            mask = np.logical_or(mask, coco.annToMask(ann)).astype(np.uint8)

        masked = img * mask[..., None]
        masked = cv2.resize(masked, IMG_SIZE)
        cv2.imwrite(str(out), masked)
else:
    print("RGB_DIR already complete. Skipping.")

# ================= STEP 2: GRAYSCALE =================
if len(list(GRAY_DIR.glob("*.jpg"))) < len(image_ids):
    print("Converting to grayscale...")
    for p in tqdm(sorted(RGB_DIR.glob("*.jpg"))):
        out = GRAY_DIR / p.name
        if out.exists(): 
            continue
        img = cv2.imread(str(p))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray = cv2.resize(gray, IMG_SIZE)
        cv2.imwrite(str(out), gray)
else:
    print("GRAY_DIR already complete. Skipping.")

# ================= STEP 3: MASKS =================
if len(list(MASK_DIR.glob("*.npy"))) < len(image_ids):
    print("Creating masks...")
    for iid in tqdm(image_ids):
        out = MASK_DIR / f"{iid}.npy"
        if out.exists():
            continue

        info = coco.loadImgs([iid])[0]
        h, w = info["height"], info["width"]

        ann_ids = coco.getAnnIds(imgIds=iid)
        mask = np.zeros((h, w), dtype=np.uint8)
        for ann in coco.loadAnns(ann_ids):
            mask = np.logical_or(mask, coco.annToMask(ann)).astype(np.uint8)

        mask = cv2.resize(mask, IMG_SIZE, interpolation=cv2.INTER_NEAREST)
        np.save(str(out), mask.astype("uint8"))
else:
    print("MASK_DIR already complete. Skipping.")

# ================= STEP 4: PAIRS =================
pairs = []
for p in sorted(GRAY_DIR.glob("*.jpg")):
    iid = p.stem
    mask = MASK_DIR / f"{iid}.npy"
    if mask.exists():
        pairs.append((str(p), str(mask)))

print("Total pairs:", len(pairs))

from sklearn.model_selection import train_test_split
train_pairs, val_pairs = train_test_split(pairs, test_size=0.1, random_state=SEED)

# ================= STEP 5: TFRecords =================
def write_tfrecord(pairs_list, out_path):
    with tf.io.TFRecordWriter(str(out_path)) as w:
        for img_path, mask_path in tqdm(pairs_list):
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, IMG_SIZE)
            img = np.repeat(img[..., None], 3, axis=-1).astype(np.uint8)

            mask = np.load(mask_path).astype(np.uint8)
            mask = cv2.resize(mask, IMG_SIZE)
            mask = mask[..., None].astype(np.uint8)

            example = tf.train.Example(features=tf.train.Features(feature={
                "image": tf.train.Feature(bytes_list=tf.train.BytesList(value=[img.tobytes()])),
                "mask":  tf.train.Feature(bytes_list=tf.train.BytesList(value=[mask.tobytes()])),
            }))
            w.write(example.SerializeToString())

if AUTO_WRITE_TFRECORD:
    if not TRAIN_TF.exists():
        write_tfrecord(train_pairs, TRAIN_TF)
    else:
        print("Train TFRecord exists.")
    if not VAL_TF.exists():
        write_tfrecord(val_pairs, VAL_TF)
    else:
        print("Val TFRecord exists.")

# ================= STEP 6: tf.data =================
AUTOTUNE = tf.data.AUTOTUNE

def parse(ex):
    feat = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "mask":  tf.io.FixedLenFeature([], tf.string),
    }
    parsed = tf.io.parse_single_example(ex, feat)
    img = tf.io.decode_raw(parsed["image"], tf.uint8)
    img = tf.reshape(img, [IMG_H, IMG_W, 3])
    img = tf.cast(img, tf.float32) / 255.0

    mask = tf.io.decode_raw(parsed["mask"], tf.uint8)
    mask = tf.reshape(mask, [IMG_H, IMG_W, 1])
    mask = tf.cast(mask, tf.float32)
    return img, mask

def make_ds(path, train=True):
    ds = tf.data.TFRecordDataset(str(path), num_parallel_reads=AUTOTUNE)
    ds = ds.map(parse, num_parallel_calls=AUTOTUNE)
    if train:
        ds = ds.shuffle(4096)
    ds = ds.batch(BATCH).prefetch(AUTOTUNE)
    return ds

train_ds = make_ds(TRAIN_TF, True)
val_ds   = make_ds(VAL_TF, False)

print("Dataset ready.")

# ================= STEP 7: MODEL =================
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import ResNet50

def build_unet_resnet50():
    base = ResNet50(weights=None, include_top=False, input_shape=(IMG_H, IMG_W, 3))
    base.load_weights(LOCAL_RESNET_WEIGHTS)
    print("Loaded LOCAL ResNet50 weights.")

    c1 = base.get_layer("conv1_relu").output
    c2 = base.get_layer("conv2_block3_out").output
    c3 = base.get_layer("conv3_block4_out").output
    c4 = base.get_layer("conv4_block6_out").output
    bn = base.get_layer("conv5_block3_out").output

    def up(x, skip, f):
        x = layers.Conv2DTranspose(f, 2, strides=2, padding="same")(x)
        x = layers.Concatenate()([x, skip])
        x = layers.Conv2D(f, 3, activation="relu", padding="same")(x)
        x = layers.Conv2D(f, 3, activation="relu", padding="same")(x)
        return x

    u1 = up(bn, c4, 256)
    u2 = up(u1, c3, 128)
    u3 = up(u2, c2, 64)
    u4 = up(u3, c1, 32)

    u5 = layers.Conv2DTranspose(16, 2, strides=2, padding="same")(u4)
    u5 = layers.Conv2D(16, 3, activation="relu", padding="same")(u5)

    out = layers.Conv2D(1, 1, activation="sigmoid", dtype="float32")(u5)
    return Model(base.input, out)

model = build_unet_resnet50()
model.summary()

# ================= STEP 8: LOSS & COMPILE =================
def dice_loss(y_true, y_pred, smooth=1e-6):
    inter = tf.reduce_sum(y_true * y_pred)
    denom = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    return 1 - (2*inter+smooth)/(denom+smooth)

loss_fn = lambda y_true, y_pred: (
    tf.keras.losses.BinaryCrossentropy()(y_true, y_pred) + dice_loss(y_true, y_pred)
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss=loss_fn,
    metrics=["accuracy"]
)

# ================= STEP 9: TRAIN =================
if TRAIN_NOW:
    print("Training...")
    history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)
    model.save("/kaggle/working/unet_final.h5", include_optimizer=False)

    print("Model saved at /kaggle/working/unet_final.h5")
else:
    print("TRAIN_NOW=False — finished preprocessing.")


In [ ]:
# Create a clean model copy without optimizer/loss/lambda
clean_model = tf.keras.models.Model(
    inputs=model.input,
    outputs=model.output
)

clean_model.save("/kaggle/working/unet_final_clean.keras", include_optimizer=False)
print("Saved clean model at /kaggle/working/unet_final_clean.keras")


In [ ]:
model = tf.keras.models.load_model("/kaggle/working/unet_final_clean.keras", compile=False)
print("Loaded clean model")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import random
import tensorflow as tf

# ================= LOAD CLEAN MODEL =================
model_path = "/kaggle/working/unet_final_clean.keras"
print("Loading model from:", model_path)

model = tf.keras.models.load_model(model_path, compile=False)
print("Model loaded successfully!")

# ================= IMAGE LOADER =================
def load_gray_image(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (256,256))
    img = np.repeat(img[..., None], 3, axis=-1)   # convert gray → 3 channels
    img = img.astype(np.float32) / 255.0
    return img

# ================= VISUALIZATION =================
num_tests = 5
plt.figure(figsize=(15, num_tests * 4))

for i in range(num_tests):
    img_path, mask_path = random.choice(val_pairs)

    # Load image
    img = load_gray_image(img_path)

    # Load mask
    mask = np.load(mask_path)
    mask = cv2.resize(mask, (256, 256))

    # Predict
    pred = model.predict(img[None, ...])[0]
    pred_bin = (pred > 0.5).astype("uint8")

    # ==== DISPLAY ====
    plt.subplot(num_tests, 3, i*3+1)
    plt.imshow(img[...,0], cmap="gray")
    plt.title("Input Image")
    plt.axis("off")

    plt.subplot(num_tests, 3, i*3+2)
    plt.imshow(mask, cmap="gray")
    plt.title("Ground Truth Mask")
    plt.axis("off")

    plt.subplot(num_tests, 3, i*3+3)
    plt.imshow(pred_bin[:,:,0], cmap="gray")
    plt.title("Predicted Mask")
    plt.axis("off")

plt.tight_layout()
plt.show()
